# 2. Quick start and Reproducing the experiment

First, you need to define your own encoder.  
In example.py, we’ve provided a simple encoder for you to start with.  
You can also replace it with your own encoder.
> ℹ️**NOTE: ** 
> The function `get_encoder_dim()` must be implemented.
> It should take the input feature dimension and simply return the output embedding dimension.

In [ ]:
class standard_code(nn.Module):

    def __init__(self, input_x ) -> None:
        super().__init__()
        # calculate mean
        self.mean = torch.mean(input_x, 0)
        # calculate sigma
        self.sigma = torch.std(input_x, dim=0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        for i, sigma in enumerate(self.sigma):
          row = x.t()[i].clone()
          row_mean = row.clone()
          row_sigma = row.clone()
          row_mean[row_mean>self.mean[i]] = 1
          row_mean[row_mean<self.mean[i]] = 0
          row_sigma[row_sigma>self.sigma[i]+self.mean[i]] = 1
          row_sigma[row_sigma<self.sigma[i]-self.mean[i]] = 1
          row_sigma[row_sigma<self.sigma[i]+self.mean[i]] = 0
          row_sigma[row_sigma>self.sigma[i]-self.mean[i]] = 0
          encode = torch.stack((row_mean,row_sigma),1)
          if ( i == 0 ):
            output = encode
          else:
            output = torch.cat( ( output, encode ), 1 )

        return output

    def get_encoder_dim(self, input_dim):
        output_dim = input_dim*2
        return output_dim



Now you can run the code!  
In example.py, simply call SCARF_Experiment() and pass in the encoder you just defined.  
Then, choose the dataset you want to experiment with and whether to apply reshaping.  

There are four dataset options:  

* `all`

* `binary`

* `multi`

* `high dim`

* `large`

For detailed descriptions of each dataset category, please refer to Appendix B of the paper.

In [ ]:
exp = scarf_exp.SCARF_Experiment(standard_code, "large", encoder_reshape=True )
exp.run()

The above describes how to replace the encoder from the original paper.  
If you just want to reproduce the experiment, you only need to modify the contents of the `config.yaml` and select the desired dataset — there’s no need to define an encoder manually.  

A sample config.yaml is provided ....